# Ch5 분석과제: 최근 가계금융복지조사 분석하기

## 데이터 불러오기 및 필요 데이터 추출

In [3]:
# 필요 패키지 로드

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
# 데이터 파일 불러오기

data = "C:/Users/narin/Desktop/data/2023_household_finance_data.csv"
df = pd.read_csv(data, encoding = 'CP949')
df.columns

Index(['조사연도', 'MD제공용_가구고유번호', '가중값', '수도권여부', '가구주_성별코드', '가구원수', '노인가구여부',
       '조손가구여부', '한부모가구여부', '다문화가구여부(2012년~)',
       ...
       '원리금상환_가계지출감소항목코드', '부채상환가능시기코드', '가구주_은퇴여부', '가구주_미은퇴_은퇴연령',
       '가구주_미은퇴_최소생활비', '가구주_미은퇴_적정생활비', '가구주_미은퇴_노후준비상황코드', '가구주_은퇴_은퇴연령',
       '가구주_은퇴_적정생활비충당여부', '가구주_은퇴_생활비재원마련코드(2014년~)'],
      dtype='object', length=160)

In [15]:
# 필요 데이터 추출

df_income = df[["조사연도", "경상소득(보완)"]]
df_income.head(5)

,조사연도,경상소득(보완)
0,2023,1655
1,2023,2995
2,2023,3001
3,2023,1871
4,2023,3603


In [17]:
df_income.rename(columns = {"조사연도":"year", "경상소득(보완)":"income"}, inplace = True) # 컬럼 이름명 변경
df_income.dropna() # 결측치 제거
df_income.head(5)

C:\Users\narin\AppData\Local\Temp\ipykernel_2448\3812693135.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_income.rename(columns = {"조사연도":"year", "경상소득(보완)":"income"}, inplace = True) # 컬럼 이름명 변경


,year,income
0,2023,1655
1,2023,2995
2,2023,3001
3,2023,1871
4,2023,3603


## 2023년 우리나라 가구의 평균 소득과 95% 신뢰구간

In [70]:
# 평균 소득
mean_income = np.round(np.mean(df_income['income']), 2)
mean_income

6194.59

In [72]:
# 표준편차
std_income = np.round(np.std(df_income['income']), 2)
std_income

6979.72

In [74]:
# 표준오차
se_income = np.round(std_income / np.sqrt(len(df_income['income'])), 2)
se_income

51.89

In [76]:
# 95% 신뢰도 신뢰구간
lower = np.round(mean_income - 1.96 * se_income, 2) # 신뢰하한
upper = np.round(mean_income + 1.96 * se_income, 2) # 신뢰상한
CI = (lower, upper)
CI

(6092.89, 6296.29)

2023년 우리나라 가구의 평균 소득은 약 6195만원이고, 95% 신뢰구간은 약 6093만원에서 6296만원까지입니다.

## 300가구 무작위 추출 후 평균 소득과 95% 신뢰구간

In [84]:
# 300가구 무작위 추출
sample_income = np.random.choice(df_income['income'], size = 300, replace = True)

In [86]:
# 평균 소득
mean_sample = np.round(np.mean(sample_income), 2)
mean_sample

5935.01

In [88]:
# 표준오차
se_sample = np.round(np.std(sample_income) / np.sqrt(300), 2)
se_sample

267.87

In [90]:
# 95% 신뢰도 신뢰구간
lower_sample = np.round(mean_sample - 1.96 * se_sample, 2) # 신뢰하한
upper_sample = np.round(mean_sample + 1.96 * se_sample, 2) # 신뢰상한
CI_sample = (lower_sample, upper_sample)
CI_sample

(5409.98, 6460.04)

무작위 추출한 300가구의 평균 소득은 약 5935만원이고, 95% 신뢰구간은 약 5410만원에서 6460만원입니다.

## 300가구 무작위추출을 1000회 반복하여 1000개의 신뢰구간을 구하고, 모평균이 신뢰구간을 벗어날 확률 구하기

In [98]:
# 300가구 무작위 추출과 신뢰구간 계산 1000회 반복

n = 1000
interval = np.zeros((n, 2))

for i in range(n):
    sample = np.random.choice(df_income['income'], size = 300, replace = True)
    mn = np.mean(sample)
    es = np.std(sample) / np.sqrt(300)
    lower = mn - 1.96 * es
    upper = mn + 1.96 * es
    interval[i][0] = lower
    interval[i][1] = upper

test = pd.DataFrame(data = interval, columns = ["lower", "upper"])
test["mean"] = np.mean(df_income['income'])
test.head()

,lower,upper,mean
0,5383.498489,6836.314844,6194.593125
1,5462.869816,6653.683518,6194.593125
2,5685.276861,7037.276472,6194.593125
3,5381.351542,6862.108458,6194.593125
4,5628.320085,6828.379915,6194.593125


In [108]:
test['test_l'] = np.where((test["lower"] > test["mean"]), 1, 0)
test['test_u'] = np.where((test["upper"] < test["mean"]), 1, 0)
test['test'] = test["test_l"] + test["test_u"]
test.head()

,lower,upper,mean,test,test_l,test_u
0,5383.498489,6836.314844,6194.593125,0,0,0
1,5462.869816,6653.683518,6194.593125,0,0,0
2,5685.276861,7037.276472,6194.593125,0,0,0
3,5381.351542,6862.108458,6194.593125,0,0,0
4,5628.320085,6828.379915,6194.593125,0,0,0


In [116]:
# 모평균이 신뢰구간을 벗어난 횟수
num = test["test"].sum()
num

64

In [118]:
# 모평균이 신뢰구간을 벗어날 확률
num / 1000

0.064